# Desafio Técnico: Pipeline RAG

## Objetivo:
Desenvolver um pipeline funcional que:
- Indexa documentos heterogêneos (PDFs com e sem texto, imagens)
- Aplique OCR quando necessário
- Gere uma base vetorial
- Implemente recuperação de informações
- Utilize uma LLM para responder perguntas com base nos
documentos recuperados

Este notebook implementa uma solução para o desafio técnico proposto. O objetivo é construir um pipeline RAG capaz de processar documentos heterogêneos,um PDF com texto e uma imagem de uma tabela de preços, e responder a perguntas em linguagem natural com base em seu conteúdo.

A solução foi projetada para ser executada como um notebook jupyter, por meio do Google Colab e utiliza a API do Groq para o componente LLM.

O pipeline consiste nas seguintes etapas:

1. Configuração do Ambiente: Instalação de dependências e configuração das chaves de API.
2. Ingestão e Extração: Carregamento dos documentos do Google Drive e extração de seu conteúdo textual. Uma lógica de "triagem" aplica extração direta para o PDF e um pipeline de OCR (Reconhecimento Óptico de Caracteres) para a imagem.
3. Indexação: O texto extraído é dividido em chunks, convertido em vetores semânticos e utiliza o FAISS como ferramenta para calculo de similaridade.
4. Recuperação e Geração: Para uma dada pergunta, o sistema recupera os chunks de texto mais relevantes do banco vetorial e os utiliza para augment um prompt enviado ao LLM da Groq (nesse caso o Llama 3 70b), que então gera uma resposta.

# Configuração do Ambiente

1. Carregamento de Dados e configuração de caminhos

Inicialmente, tem a montagem do Google Drive na sessão do Colab. Em seguida, a definição dos caminhos exatos para os arquivos PDF e de imagem.

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Para execução do arquivo localmente, terá que ter osos arquivos em uma pasta docs dentro do diretório que tem o notebook.

In [2]:
PDF_PATH = '/content/drive/MyDrive/nuven/CÓDIGO DE OBRAS.pdf'
IMAGE_PATH = '/content/drive/MyDrive/nuven/tabela.webp'

# PDF_PATH = '/docs/CÓDIGO DE OBRAS.pdf'
# IMAGE_PATH = '/docs/tabela.webp'

2. Instalação de Dependências

Instalando todas as bibliotecas necessárias para o projeto, que não estão ainda no google colab

In [3]:
# Instalação das bibliotecas Python necessárias
!pip install -qU \
  langchain-groq \
  pymupdf \
  pytesseract \
  langchain_community \
  langchain-huggingface \
  faiss-cpu

# Instalação do Tesseract OCR no ambiente do Colab
!sudo apt-get install -y tesseract-ocr tesseract-ocr-por

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.1/24.1 MB 58.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 38.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 20.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 131.1/131.1 kB 11.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 4.3 MB/s eta 0:00:00
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
tesseract-ocr is already the newest version (4.1.1-2.1build1).
The following NEW packages will be installed:
  tesseract-ocr-por
0 upgraded, 1 newly installed, 0 to remove and 35 not upgraded.
Need to get 856 kB of archives.
After this operation, 1,998 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr-por all 1:4.00~git30-7274cfa-1.1 [856 kB]
Fetched 856 kB in 1s

3. Configuração da chave de API e importações

As chaves de API do huggingface e do groq estão sendo gerenciadas pelo secrets no colab, caso o notebook seja rodado em um contexto mais local deve utilizar o load_env e no diretório deve haver o arquivo .env com as chaves.

In [ ]:
import os
import re
import fitz
import pytesseract
import numpy as np
from google.colab import userdata
from PIL import Image, ImageEnhance

from langchain_groq import ChatGroq
from langchain.prompts import PromptTemplate
from langchain.docstore.document import Document
from langchain_community.vectorstores import FAISS
from langchain_huggingface import HuggingFaceEmbeddings
from langchain.schema.runnable import RunnablePassthrough
from langchain.schema.output_parser import StrOutputParser
from langchain.text_splitter import RecursiveCharacterTextSplitter

# Configuração da chave de API do Groq a partir dos secrets do Colab
os.environ['GROQ_API_KEY'] = userdata.get('GROQ_API_KEY')
os.environ['HF_TOKEN'] = userdata.get('HF_TOKEN')

# from dotenv import load_dotenv
# load_dotenv()


# Extração de Conteúdo

Esta seção implementa a extração de texto dos documentos de origem. Adotando uma abordagem de "triagem" para lidar com os diferentes formatos de arquivo:

Para o PDF (CÓDIGO DE OBRAS.pdf): Utilizando da biblioteca PyMuPDF/fitz para extrair o texto nativo página por página.

Para a Imagem (tabela.webp): Implementação de um pipeline de OCR. A imagem é lida com PIL e o texto é extraído com pytesseract. O passo mais crítico aqui é a reconstrução da estrutura tabular. A saída bruta do OCR é pós-processada para transformar cada linha da tabela em uma string de texto estruturada e legível, buscando preservar a relação entre código, artigo e preços.

In [ ]:
def structure_ocr_text(ocr_text):
  """
  Pós-processa o texto bruto do OCR de uma tabela para reconstruir sua estrutura.
  Transforma cada linha da tabela em uma string descritiva.
  """
  structured_lines = []
  lines = ocr_text.strip().split('\n')

  header_keywords = ['código', 'artigo', 'unidade', 'preço s iva', 'preço c iva']
  start_index = 0
  for i, line in enumerate(lines):
    if any(keyword in line.lower() for keyword in header_keywords):
      start_index = i + 1
      break

  # Expressão regular para capturar os componentes de cada linha
  line_regex = re.compile(r'(\S+)\s+(.*?)\s+(Un|Cx)\s+([\d,]+)\s*€?\s+([\d,]+)\s*€?')

  for line in lines[start_index:]:
    line = line.strip()
    if not line:
      continue

    match = line_regex.match(line)
    if match:
      codigo, artigo, und, preco_sem_iva, preco_com_iva = match.groups()

      # Limpeza e formatação
      artigo = artigo.strip()
      
      structured_line = (
        f"Item: {artigo.strip()} | "
        f"Código: {codigo.strip()} | "
        f"Unidade: {und.strip()} | "
        f"Preço sem IVA: {preco_sem_iva.strip()} € | "
        f"Preço com IVA: {preco_com_iva.strip()} €."
      )
      structured_lines.append(structured_line)
    else:
      structured_lines.append(f"Informação adicional: {line}")

  return "\n".join(structured_lines)


def extract_text_from_image(file_path):
  """Realiza OCR em uma imagem, estrutura o texto e retorna como um Document."""
  try:
    image = Image.open(file_path).convert('L')
    image = ImageEnhance.Contrast(image)
    image = image.enhance(2.0)
    ocr_text = pytesseract.image_to_string(image, lang='por')

    structured_text = structure_ocr_text(ocr_text)

    if not structured_text.strip():
      print(f"AVISO: Nenhum texto estruturado pôde ser extraído da imagem: {os.path.basename(file_path)}")
      return []

    print(f"Texto extraído e estruturado da imagem: {os.path.basename(file_path)}")

    return [Document(
      page_content=structured_text,
      metadata={"source": os.path.basename(file_path), "page": 1}
    )]

  except Exception as e:
    print(f"Erro ao processar a imagem {file_path}: {e}")
    return []

def extract_text_from_pdf(file_path):
  """Extrai texto de um arquivo PDF com metadados de página."""
  try:
    doc = fitz.open(file_path)
    documents = []

    for page_num, page in enumerate(doc):
      text = page.get_text()
      if text.strip():
        documents.append(Document(
          page_content=text,
          metadata={"source": os.path.basename(file_path), "page": page_num + 1}
        ))

    print(f"Extraídas {len(documents)} páginas do PDF: {os.path.basename(file_path)}")

    return documents

  except Exception as e:
    print(f"Erro ao processar PDF {file_path}: {e}")
    return []


In [6]:
all_documents = []
file_paths = [PDF_PATH, IMAGE_PATH]

for path in file_paths:
    if not os.path.exists(path):
        print(f"Arquivo não encontrado, pulando: {path}")
        continue

    if path.lower().endswith('.pdf'):
        all_documents.extend(extract_text_from_pdf(path))
    elif path.lower().endswith(('.png', '.jpg', '.jpeg', '.webp')):
        all_documents.extend(extract_text_from_image(path))

print(f"\nTotal de documentos processados para indexação: {len(all_documents)}")

Extraídas 85 páginas do PDF: CÓDIGO DE OBRAS.pdf
Texto extraído e estruturado da imagem: tabela.webp

Total de documentos processados para indexação: 86


# Indexação
Com o texto extraído e pré-processado, o próximo passo é prepará-lo para a busca semântica.

1. Divisão de Texto (Chunking)

Para que o modelo de embedding e o LLM possam procesar os documentos melhor, o texto deve ser dividido em chunks. Utilizando o RecursiveCharacterTextSplitter, que tenta preservar a coesão semântica ao dividir preferencialmente por parágrafos e depois por linhas. Um chunk_size de 1200 caracteres com uma sobreposição de 300 para que que o contexto não seja perdido nas fronteiras dos chunks.

2. Embeddings e Armazenamento Vetorial

Para cada chunk de texto é então feito o embedding, transformando o texto em vetores numericos por meio de um modelo da biblioteca sentence-transformers.Todos os vetores e seus textos correspondentes são carregados em um índice FAISS, que permite uma busca de similaridade. Este índice atuará como retriever.

In [7]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1200,
    chunk_overlap=300,
    length_function=len,
    separators=["\n\n", "\n", "(?<=\. )", " ", ""]
)

model_name = "sentence-transformers/all-MiniLM-L6-v2"
embeddings = HuggingFaceEmbeddings(model_name=model_name)
chunks = text_splitter.split_documents(all_documents)
print(f"\nO conteúdo foi dividido em {len(chunks)} chunks.")

vectorstore = FAISS.from_documents(chunks, embeddings)

retriever = vectorstore.as_retriever(
    search_type="mmr",
    search_kwargs={'k': 10, 'fetch_k':30}
)

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]


O conteúdo foi dividido em 246 chunks.


# Recuperação e Geração
Esta é a fase final, onde o objetivo será responder as perguntas do usuário com o contexto dos arquivos. O processo funciona da seguinte forma:

1. Consulta: O usuário insere uma pergunta em linguagem natural.
2. Recuperação: A pergunta é vetorizada, e o retriever FAISS busca no índice os chunks de texto mais semanticamente similares.
3. Augmentation: Os chunks recuperados são inseridos em um template de prompt. Este prompt é cuidadosamente projetado para instruir o LLM a basear sua resposta exclusivamente no contexto fornecido, agindo como um especialista nos documentos. Isso minimiza o risco de o modelo "alucinar" ou usar conhecimento externo.
4. Geração: O prompt completo é enviado para a API do Groq, que gera a resposta final.

In [8]:
# Definir o modelo de chat da Groq
llm = ChatGroq(model_name="llama3-70b-8192")

# Template do prompt
template = """
Você é um assistente especializado em regulamentos de construção e materiais de
construção, baseado estritamente nos documentos fornecidos. Sua tarefa é
responder à pergunta usando apenas o contexto abaixo.

Se a informação não estiver no contexto, responda: "Com base nos documentos
fornecidos, não foi possível encontrar uma resposta para a sua pergunta."
Não invente informações. Cite a fonte (nome do arquivo e página, se disponível)
da sua resposta.

Contexto:
{context}

Pergunta:
{question}

Resposta:
"""

prompt = PromptTemplate.from_template(template)

# Criação da RAG Chain
rag_chain = (
  {"context": retriever,
   "question": RunnablePassthrough()}
  | prompt
  | llm
  | StrOutputParser()
  )

# função para invocar a cadeia e imprimir a resposta de forma limpa
def ask_question(query):
  print(f"--- Pergunta ---\n{query}\n")
  answer = rag_chain.invoke(query)
  print(f"--- Resposta ---\n{answer}")

## Demonstração

Teste do pipeline com um conjunto de perguntas para avaliar a capacidade de extração de informações dos documentos.

In [9]:
# Teste 1: Consulta que requer informação do PDF
query1 = "Quais são as dimensões e áreas mínimas para os dois primeiros quartos de uma residência, conforme o anexo 01?"
ask_question(query1)

--- Pergunta ---
Quais são as dimensões e áreas mínimas para os dois primeiros quartos de uma residência, conforme o anexo 01?

--- Resposta ---
Com base nos documentos fornecidos, a resposta para a sua pergunta pode ser encontrada no documento "CÓDIGO DE OBRAS.pdf", página 84, Anexo 01. Segundo o anexo, as dimensões e áreas mínimas para os dois primeiros quartos de uma residência são:

* 1º Quarto: 2,20m (circular mínimo inscrito) e 8,00m² (área mínima)
* 2º Quarto: 2,20m (circular mínimo inscrito) e 8,00m² (área mínima)

Fonte: Document(id='1b0660b9-4dc1-4eb8-bb77-4828a50317a2', metadata={'source': 'CÓDIGO DE OBRAS.pdf', 'page': 84})


In [10]:
# Teste 2: Consulta que requer informação do PDF
query2 = "Qual a ocupação máxima do passeio permitida para tapumes e andaimes?"
ask_question(query2)

--- Pergunta ---
Qual a ocupação máxima do passeio permitida para tapumes e andaimes?

--- Resposta ---
Com base nos documentos fornecidos, a resposta pode ser encontrada no Art. 36 do CÓDIGO DE OBRAS.pdf, página 18: "Tapumes e andaimes não poderão ocupar mais do que a metade da largura do passeio sendo que, no mínimo, 0,80m serão mantidos livres para o fluxo de pedestres."

Portanto, a ocupação máxima do passeio permitida para tapumes e andaimes é de metade da largura do passeio.


In [11]:
# Teste 3: Consulta à tabela de preços (tabela.webp)
query3 = "Qual o preço com IVA do 'Extintor (3 kg)'?"
ask_question(query3)

--- Pergunta ---
Qual o preço com IVA do 'Extintor (3 kg)'?

--- Resposta ---
Com base nos documentos fornecidos, a resposta é: 53,38 €. (Fonte: tabela.webp, página 1)


In [15]:
# Teste 4: Consulta à tabela de preços (tabela.webp)

query4 = "Qual os preços com e sem IVA de um 'Rebites em inox (caixa 500)'?"
ask_question(query4)

--- Pergunta ---
Qual os preços com e sem IVA de um 'Rebites em inox (caixa 500)'?

--- Resposta ---
Com base nos documentos fornecidos, a resposta é: Preço sem IVA: 70,53 € | Preço com IVA: 86,15 €. (Fonte: tabela.webp, página 1)


In [16]:
# Teste 5: Consulta sobre um tópico que não está nos documentos
# Teste da capacidade do modelo de seguir a instrução de "não inventar".
query5 = "Qual é a norma para a instalação de painéis solares no município de Eusébio?"
ask_question(query5)

--- Pergunta ---
Qual é a norma para a instalação de painéis solares no município de Eusébio?

--- Resposta ---
Com base nos documentos fornecidos, não foi possível encontrar uma resposta para a sua pergunta.
